In [1]:
import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

# My data
class MyWebData(btfeeds.GenericCSVData):
    params = (
        ('fromdate', datetime.datetime(2020, 11, 1)),
        ('todate', datetime.datetime(2020, 11, 30)),
        ('timeframe', bt.dataseries.TimeFrame.Minutes) , 
        
        ('nullvalue', 0.0),
        ('dtformat', ('%Y/%m/%d')),
        ('tmformat', ('%H:%M:%S')),
        
        ('datetime', 0),
        ('time', 1),
        ('high', 3),
        ('low', 4),
        ('open', 2),
        ('close', 5),
        ('volume', 6),
        ('openinterest', -1)
        
    )

data = MyWebData(dataname='../data/website/TXF20110101_20201231.csv')

In [2]:
def create_cerebro(s):
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    # cerebro.addstrategy(MyStrategy, period=90)
    cerebro.addstrategy(s)

    # Add the Data Feed to Cerebro
    # cerebro.adddata(data)
    cerebro.resampledata(data, timeframe=bt.dataseries.TimeFrame.Minutes, compression=5)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # set sizer
    cerebro.addsizer(bt.sizers.SizerFix, stake=3)

    # Print out the starting conditions
    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())

    cerebro.addanalyzer(btanalyzers.DrawDown, _name="DrawDown")
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name="TradeAnalyzer")
    cerebro.addanalyzer(btanalyzers.Transactions, _name="Transactions")
    
    cerebro.broker.setcommission(commission=2.0, margin=800)

    return cerebro

In [3]:
import datetime

import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.indicators as btind


class MeanAverageBaseIndicator(bt.Indicator):
    params = (('value', 5),)
    
    def __init__(self) -> None:
        self.movav5 = btind.SimpleMovingAverage(self.data, period=5)
        self.movav10 = btind.SimpleMovingAverage(self.data, period=10)
        self.movav20 = btind.SimpleMovingAverage(self.data, period=20)
        self.movav60 = btind.SimpleMovingAverage(self.data, period=60)

class LongArrange(MeanAverageBaseIndicator):
    lines = ('arrange',)
    def __init__(self) -> None:
        super().__init__()
        self.a1 = self.movav5 > self.movav10
        self.a2 = self.movav10 > self.movav20
        self.a3 = self.movav10 > self.movav60
        
        self.c1 = bt.And(self.a1, self.a2)
        self.c2 = bt.And(self.a2, self.a3)
        
        self.lines.arrange = bt.And(self.c1, self.c2)

class ShortArrange(MeanAverageBaseIndicator):
    lines = ('arrange',)
    def __init__(self) -> None:
        super().__init__()
        self.a1 = self.movav5 <= self.movav10
        self.a2 = self.movav10 <= self.movav20
        self.a3 = self.movav10 <= self.movav60
        
        self.c1 = bt.And(self.a1, self.a2)
        self.c2 = bt.And(self.a2, self.a3)
        
        self.lines.arrange = bt.And(self.c1, self.c2)

class ShortCrossLong(MeanAverageBaseIndicator):
    lines = ('cross', )

    def __init__(self) -> None:
        super().__init__()
        self.long_arrange = LongArrange()

    def next(self) -> None:
        over_ma5 = self.data.close[0] > self.movav5[0]
        # print(f'Indicator: {over_ma5} close:{self.data.close[0]} ma5:{self.movav5[0]}')
        self.lines.cross[0] = over_ma5 and self.long_arrange[0]
        # print(f'Signal: {self.lines.cross[0]}')

class BaseStrategy(bt.Strategy):
    # only ma across
    params = dict(period=20, stake=1, onlylong=True)

    def __init__(self):
        self.buy_signal = ShortCrossLong()
        self.sell_signal = ShortCrossLong()

    def log(self, txt, dt=None):
        """Logging function for this strategy"""
        dt = dt or f"{self.datas[0].datetime.date(0)} {self.datas[0].datetime.time(0)}"
        print(f"{dt}\tTxt:{txt}")

    def next(self):

        # if self.signal > 0.0:  # cross upwards
        if self.signal:  # cross upwards
            if self.position:
                # self.log('CLOSE SHORT , %.2f' % self.data.close[0])
                self.close()

            # self.log("BUY CREATE , %.2f" % self.data.close[0])
            self.buy(size=self.p.stake)

        # elif self.signal < 0.0:
        else:
            if self.position:
                # self.log("CLOSE LONG , %.2f" % self.data.close[0])
                self.close()

            if not self.p.onlylong:
                # self.log("SELL CREATE , %.2f" % self.data.close[0])
                self.sell(size=self.p.stake)

class ArrangeStrategy(BaseStrategy):
    params = dict(period=20, stake=1, onlylong=True)

    def __init__(self):
        self.s1 = ShortCrossLong()
        self.s2 = LongArrange()
        self.signal = bt.And(self.s1, self.s2)

In [4]:
start = datetime.datetime.now()

c = create_cerebro(ArrangeStrategy)
r = c.run()

# Print out the final result
print("Final Portfolio Value: %.2f" % c.broker.getvalue())
print(f'Spent: {datetime.datetime.now() - start}')

Starting Portfolio Value: 100000.00
Final Portfolio Value: 96814.00
Spent: 0:01:09.665986


In [5]:
print(f'Spent: {datetime.datetime.now() - start}')


Spent: 0:01:09.671819


In [7]:
c.broker.getvalue()
# r.analyzers.TradeAnalyzer.get_analysis()

96814.0

In [99]:
start = datetime.datetime.now()

c1 = create_cerebro(BaseStrategy)
r1 = c1.run()

# Print out the final result
print("Final Portfolio Value: %.2f" % c1.broker.getvalue())

print(f'Spent: {datetime.datetime.now() - start}')

start = datetime.datetime.now()

c2 = create_cerebro(ArrangeStrategy)
r2 = c2.run()

# Print out the final result
print("Final Portfolio Value: %.2f" % c2.broker.getvalue())

print(f'Spent: {datetime.datetime.now() - start}')

Starting Portfolio Value: 100000.00
Final Portfolio Value: 91288.00
Spent: 0:04:30.749720
Starting Portfolio Value: 100000.00
Final Portfolio Value: 104522.00
Spent: 0:04:56.724747


In [ ]:
r1[0].analyzers.TradeAnalyzer.get_analysis()

In [ ]:
r2[0].analyzers.TradeAnalyzer.get_analysis()